In [2]:
import xlwings as xw
import random
import time
import keyboard

In [ ]:
wb = xw.Book()
sheet = wb.sheets.active
ROWS, COLS = 20, 10

In [6]:
COLOR_MAP = {
    0: (255, 255, 255),
    1: (255, 0, 0),
    2: (0, 0, 255)
}

In [7]:
SHAPES = {
    "I": [(0, 0), (0, 1), (0, 2), (0, 3)],
    "O": [(0, 0), (0, 1), (1, 0), (1, 1)],
    "T": [(0, 1), (1, 0), (1, 1), (1, 2)],
    "L": [(0, 0), (1, 0), (2, 0), (2, 1)]
}

In [8]:
current_piece = random.choice(list(SHAPES.keys()))
current_coordinates = SHAPES[current_piece]

In [9]:
for i in range(ROWS):
    for j in range(COLS):
        value = sheet.range(i + 1, j + 1).value
        color = COLOR_MAP.get(value)
        sheet.range(i + 1, j + 1).color = color

In [10]:
for r,c in current_coordinates:
    sheet.range(r + 1, c + 4).value = 1

In [11]:
def rotate():
    global current_coordinates

    pivot = current_coordinates[0]
    pivot_r, pivot_c = pivot

    new_coordinates = [(pivot_r + (c - pivot_c), pivot_c - (r - pivot_r)) for r, c in current_coordinates]

    for r, c in new_coordinates:
        if r < 0 or r >= ROWS or c < 0 or c >= COLS or sheet.range(r + 1, c + 4).value == 1:
            return

    for r, c in current_coordinates:
        sheet.range(r + 1, c + 4).value = 0

    for r, c in new_coordinates:
        sheet.range(r + 1, c + 4).value = 1

    current_coordinates = new_coordinates

In [12]:
def clear_full_rows():
    for r in range(ROWS):
        if all(sheet.range(r + 1, 4, r + 1, 4 + COLS - 1).value):
            sheet.range(r + 1, 4, r + 1, 4 + COLS - 1).value = [0] * COLS
            for move_r in range(r, 0, -1):
                above_row = sheet.range(move_r, 4, move_r, 4 + COLS - 1).value
                sheet.range(move_r + 1, 4, move_r + 1, 4 + COLS - 1).value = above_row
            sheet.range(1, 4, 1, 4 + COLS - 1).value = [0] * COLS

In [13]:
def place_piece():
    for r, c in current_coordinates:
        sheet.range(r + 1, c + 4).value = 1
    clear_full_rows()

In [14]:
def spawn_new_piece():
    global current_piece, current_coordinates

    for r, c in current_coordinates:
        sheet.range(r + 1, c + 4).value = 0

    current_piece = random.choice(list(SHAPES.keys()))
    current_coordinates = SHAPES[current_piece]

    for r, c in current_coordinates:
        if sheet.range(r + 1, c + 4).value == 1:
            print("GAME OVER!")
            exit()

    for r, c in current_coordinates:
        sheet.range(r + 1, c + 4).value = 1

In [15]:
def clear_full_rows():
    for r in range(ROWS):
        if all(sheet.range(r + 1, 4, r + 1, 4 + COLS - 1).value):
            sheet.range(r + 1, 4, r + 1, 4 + COLS - 1).value = [0] * COLS

            for move_r in range(r, 0, -1):
                above_row = sheet.range(move_r, 4, move_r, 4 + COLS - 1).value
                sheet.range(move_r + 1, 4, move_r + 1, 4 + COLS - 1).value = above_row
            
            sheet.range(1, 4, 1, 4 + COLS - 1).value = [0] * COLS

In [16]:
def down():
    global current_coordinates, current_piece

    for r, c in current_coordinates:
        if r + 1 >= ROWS or sheet.range(r + 2, c + 4).value == 1:
            place_piece()
            spawn_new_piece()
            return False

    for r, c in current_coordinates:
        sheet.range(r + 1, c + 4).value = 0

    new_coordinates = [(r + 1, c) for r, c in current_coordinates]
    for r, c in new_coordinates:
        sheet.range(r + 1, c + 4).value = 1

    current_coordinates = new_coordinates
    return True

def left():
    global current_coordinates
    for r, c in current_coordinates:
        if c - 1 < 0 or sheet.range(r + 1, c + 3).value == 1:
            return

    for r, c in current_coordinates:
        sheet.range(r + 1, c + 4).value = 0

    new_coordinates = [(r, c - 1) for r, c in current_coordinates]
    for r, c in new_coordinates:
        sheet.range(r + 1, c + 4).value = 1

    current_coordinates = new_coordinates

def right():
    global current_coordinates

    for r, c in current_coordinates:
        if c + 1 >= COLS or sheet.range(r + 1, c + 5).value == 1:
            return

    for r, c in current_coordinates:
        sheet.range(r + 1, c + 4).value = 0

    new_coordinates = [(r, c + 1) for r, c in current_coordinates]
    for r, c in new_coordinates:
        sheet.range(r + 1, c + 4).value = 1

    current_coordinates = new_coordinates

keyboard.add_hotkey('left', left)
keyboard.add_hotkey('right', right)
keyboard.add_hotkey('down', down)
keyboard.add_hotkey('up', rotate)

while down():
    time.sleep(0.5)

Exception in thread Thread-4 (process):
Traceback (most recent call last):
  File "C:\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\LENOVO\Desktop\personal_projects\Tetris_in_excel\main\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\LENOVO\Desktop\personal_projects\Tetris_in_excel\main\Lib\site-packages\keyboard\_generic.py", line 58, in process
    if self.pre_process_event(event):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LENOVO\Desktop\personal_projects\Tetris_in_excel\main\Lib\site-packages\keyboard\__init__.py", line 218, in pre_process_event
    callback(event)
  File "C:\Users\LENOVO\Desktop\personal_projects\Tetris_in_excel\main\Lib\site-packages\keyboard\__init__.py", line 649, in <lambda>
    handler = lambda e: (event_type == KEY_DOWN and e.event_t

TypeError: Sheet.range() takes from 2 to 3 positional arguments but 5 were given